# Title
[]()

In [1]:

import pandas as pd
import sys
import os
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from file_functions import *
# import time
import re

# from response_processing import *
# from article_processing import create_text_dict_from_folder
# import traceback
from file_functions import *

from IPython import display
# import scrapy
# from scrapy.crawler import CrawlerRunner
# from crochet import setup, wait_for

In [2]:
# set the option to wrap text within cells
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Load scraped article_dict

In [3]:
path = '../web_articles/2023-06-21'
article_dict = {
    0: loadpickle('scraped_articles_dict_2023-06-21_2136.sav', path)
}

	Time completed: 2023-06-24 22:05:37.349123
Dictionary keys: [1.0, 1.01, 1.02, 1.03, 1.04, 1.05, 1.06, 1.07, 1.08, 1.09, 1.1, 1.11, 1.12, 1.13, 1.14, 1.15, 1.16, 1.17, 1.18, 1.19, 1.2, 1.21, 1.22, 1.23, 1.24, 1.25, 1.26, 1.27, 1.28, 1.29, 1.3, 1.31, 1.32, 1.33, 1.34, 1.35, 1.36, 1.37, 1.38, 1.39, 1.4, 3.0, 3.01, 3.02, 3.03, 3.04, 3.05, 3.06, 3.07, 3.08, 3.09, 3.1, 3.11, 3.12, 3.13, 3.14, 3.15, 3.16, 3.17, 3.18, 3.19, 3.2, 3.21, 3.22, 3.23, 3.24, 3.25, 3.26, 1.41, 1.42, 1.43, 1.44, 1.45, 1.46, 1.47, 1.48, 1.49, 3.27, 3.28, 3.29, 0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 2.0, 2.01, 2.02, 2.03, 2.04, 2.05, 2.06, 2.07, 2.08, 2.09]


In [4]:
next(iter(article_dict[0].values()))

{'journal': 'BMJ Open',
 'title': 'Recommendations for hand hygiene in community settings: a scoping review of current international guidelines',
 'url': 'http://bmjopen.bmj.com/cgi/content/short/13/6/e068887?rss=1',
 'text': '\n<p class="logo-bmj-journals"><a href="http://journals.bmj.com/" title="BMJ Journals"><img src="/sites/default/themes/bmjj/img/logos/logo-bmj-journals.svg" title="BMJ Journals" alt="BMJ Journals"></a></p>\n<h3>Log in using your username and password</h3>\n<h2 class="pane-title"><span class="icon-search"></span></h2>\n<h2 class="pane-title"><span class="icon-menu"></span></h2>\n<h2 class="pane-title">Main menu</h2>\n<h3>Log in using your username and password</h3>\n<h2 class="element-invisible">You are here</h2>\n<p><span class="icon-article-text"></span>Article Text</p>\n<p><a class="article-pdf-download" href="/content/bmjopen/13/6/e068887.full.pdf" target="new"><img alt="Download PDF" src="/sites/default/themes/bmjj/img/icon-pdf.png"><strong>PDF</strong></a></

# `create_partial_article_dict`

In [5]:
def create_partial_article_dict(article_dict, n_articles=2, journals='all'):
    """
    Creates a partial article dictionary from the full article dictionary.
    
    Args:
        article_dict (dict): The full article dictionary.
        n_articles (int, optional): The number of articles per journal to include in the partial dictionary.
            Defaults to 2.
        journals ('all', int, or list, optional): The integers of the journals to include in the partial dictionary.
            Defaults to 'all'.
    
    Returns:
        dict: A partial article dictionary.
    """
    if journals == 'all':
        journals = list(set([key//1 for key in article_dict.keys()]))
    elif (type(journals) == float) or (type(journals) == int):
        journals = [journals]
    article_dict = {
        key: article_dict[key] for key in article_dict.keys() if \
        (key//1 in journals) and (key - int(key) < n_articles/100)
        }
    print(f'Keys for article_dict: {[key for key in sorted(article_dict.keys())]}')
    journals = [journal for journal in set([key["journal"] for key in article_dict.values()])]
    print('Journals:')
    for journal in journals:
        print(f'\t{journal}')
    return article_dict


text_id = 0
partial_article_dict = {text_id: create_partial_article_dict(article_dict[0], n_articles=5, journals=2)}

Keys for article_dict: [2.0, 2.01, 2.02, 2.03, 2.04, 2.05]
Journals:
	Journal of Medical Internet Research


# text_dict_from_web: From previous notebook

In [6]:
from IPython import display

def trim_text(text, article_regex=None, abs_regex=None):
    if article_regex==None:
        article_regex = '.*<h2>Abstract</h2>.*(?:Introduction.*)?(<h2.*?>Introduction</h2>.*References)<.*' 
        abs_regex = '.*(<h2>Abstract</h2>.*(?:Introduction.*)?)<h2.*?>Introduction</h2>.*References<.*' 
    try:
        body = re.search(article_regex, text, re.DOTALL).group(1)
        abstract = re.search(abs_regex, text, re.DOTALL).group(1)
    except Exception as error: 
        exc_type, exc_obj, tb = sys.exc_info()
        file = tb.tb_frame
        lineno = tb.tb_lineno
        filename = file.f_code.co_filename
        print(f'\tAn error occurred on line {lineno} in {filename}: {error}')    
        print('\t\tUnable to parse article text')
        body = text 
        abstract = text 
    try:
        article_display = display.HTML(body)
        abs_display = display.HTML(abstract)
    except Exception as error: 
        exc_type, exc_obj, tb = sys.exc_info()
        file = tb.tb_frame
        lineno = tb.tb_lineno
        filename = file.f_code.co_filename
        print(f'\tAn error occurred on line {lineno} in {filename}: {error}')    
        print('\t\tUnable to create HTML display')
        article_display = f'<p>{body}</p>'
        abs_display = f'<p>{abstract}</p>'
    processed_article = {
        'abstract': abstract,
        'body': body,
    }
    display_dict = {
        'article_display': article_display,
        'abs_display': abs_display
    }
    return processed_article, display_dict

def text_dict_from_web(article_dict, header=(2,4), to_display=0.01,
        article_regex_str=r'.*<h\d.*?>Abstract</h\d>.*(?:Introduction.*)?(<h\d.*?>Introduction</h\d>.*References)<.*',
        abs_regex_str=r'.*(<h\d.*?>Abstract</h\d>.*(?:Introduction.*)?)<h\d.*?>Introduction</h\d>.*References<.*'
        ):
    """
    Create a text dictionary from a dictionary containing web-scraped articles.

    Parameters:
        article_dict (dict): Values of each dictionary item are a dictionary representing the data from a 
            single article: 'url', 'text', and 'title'.

    Returns:
        text_dict: Dictionary where each item is a string of the text of an article, starting with the title.
    """
    if type(header) == int:
        header = str(header) 
    else :
        header = rf"[{''.join([str(h) for h in range(header[0], header[-1]+1)])}]"
    print(rf'header: {header}')
    article_regex_str = article_regex_str.replace('\d', header)
    abs_regex_str = abs_regex_str.replace('\d', header)
    article_regex = rf'{article_regex_str}'
    abs_regex = rf'{abs_regex_str}'
    print(f'Regex patterns: \n\t{article_regex}\n\t{abs_regex}')
    text_dict = dict()
    display_dict = dict()
    if (type(to_display) == int) or (type(to_display) == float):
        to_display = [to_display] 
    for article_key in article_dict:
        if (article_key +1) - (article_key +1) //1 == 0: # if integer
            print(f'Journal: {article_dict[article_key]["journal"]} {article_key}')
        trimmed_text, display = trim_text(article_dict[article_key]['text'], article_regex, abs_regex)
        text_dict[article_key] = {
            'title': article_dict[article_key]['title'],
            'body': f"{article_dict[article_key]['title']}\n\n{trimmed_text['body']}",
            'abstract': trimmed_text['abstract'],
        }
        if (to_display == 'all') or (to_display == None) or (article_key in to_display):
            display_dict[article_key] = {
                'abstract': display['abs_display'],
                'body': display['article_display']
            }
    print(f'text_dict keys: {[key for key in text_dict.keys()]}')
    return text_dict, display_dict

def display_html(display_dict, type='abstract'):
    """
    Display the HTML from the dictionary of HTML displays.
    """
    print()
    for text in display_dict:
        print('************************************* Start *************************************')
        display.display(display_dict[text][type])


root_display_dict = dict()
text_dict = dict()
iteration = 0
regex = r'(.*)'
text_dict[iteration], display_dict = text_dict_from_web(
    partial_article_dict[text_id], to_display='all', header=(2,4), article_regex_str=regex, abs_regex_str=regex)
root_display_dict[iteration] = display_dict
display_html(display_dict, type='abstract')


header: [234]
Regex patterns: 
	(.*)
	(.*)
Journal: Journal of Medical Internet Research 2.0
text_dict keys: [2.0, 2.01, 2.02, 2.03, 2.04, 2.05]

************************************* Start *************************************


************************************* Start *************************************


************************************* Start *************************************


************************************* Start *************************************


************************************* Start *************************************


************************************* Start *************************************


# Iteration 1

In [7]:
article_regex = '.*<h2>Abstract</h2>.*(?:Introduction.*)?(<h2.*?>Introduction</h2>.*References)<.*' 
abs_regex = '.*(<h2>Abstract</h2>.*(?:Introduction.*)?)<h2.*?>Introduction</h2>.*References<.*' 
article_regex = '.*<h[234].*?>Abstract</h[234]>.*(?:Introduction.*)?(<h[234].*?>Introduction</h[234]>.*References)<.*'

re.sub(r'.*(Introduction).*', r'\1', article_regex)

'Introduction'

In [8]:
from IPython import display

def trim_text(text, article_regex=None, abs_regex=None):
    if article_regex==None:
        article_regex = '.*<h2>Abstract</h2>.*(?:Introduction.*)?(<h2.*?>Introduction</h2>.*References)<.*' 
        abs_regex = '.*(<h2>Abstract</h2>.*(?:Introduction.*)?)<h2.*?>Introduction</h2>.*References<.*' 

    intro_header = re.sub(r'.*(Introduction).*', r'\1', article_regex)
    
    print(f'intro_header: {intro_header}')
    if intro_header in text:
        try:
            body = re.search(article_regex, text, re.DOTALL).group(1)
            abstract = re.search(abs_regex, text, re.DOTALL).group(1)
        except Exception as error: 
            exc_type, exc_obj, tb = sys.exc_info()
            file = tb.tb_frame
            lineno = tb.tb_lineno
            filename = file.f_code.co_filename
            print(f'\tAn error occurred on line {lineno} in {filename}: {error}')    
            print('\t\tUnable to parse article text')
            body = text 
            abstract = text 
        try:
            article_display = display.HTML(body)
            abs_display = display.HTML(abstract)
        except Exception as error: 
            exc_type, exc_obj, tb = sys.exc_info()
            file = tb.tb_frame
            lineno = tb.tb_lineno
            filename = file.f_code.co_filename
            print(f'\tAn error occurred on line {lineno} in {filename}: {error}')    
            print('\t\tUnable to create HTML display')
            article_display = f'<p>{body}</p>'
            abs_display = f'<p>{abstract}</p>'
    else:
        print('No introduction header found')
        body = text 
        abstract = text 
        article_display = f'<p>{body}</p>'
        abs_display = f'<p>{abstract}</p>'
    processed_article = {
        'abstract': abstract,
        'body': body,
    }
    display_dict = {
        'article_display': article_display,
        'abs_display': abs_display
    }
    return processed_article, display_dict

def text_dict_from_web(article_dict, header=(2,4), to_display=0.01,
        article_regex_str=r'.*<h\d.*?>Abstract</h\d>.*(?:Introduction.*)?(<h\d.*?>Introduction</h\d>.*References)<.*',
        abs_regex_str=r'.*(<h\d.*?>Abstract</h\d>.*(?:Introduction.*)?)<h\d.*?>Introduction</h\d>.*References<.*'
        ):
    """
    Create a text dictionary from a dictionary containing web-scraped articles.

    Parameters:
        article_dict (dict): Values of each dictionary item are a dictionary representing the data from a 
            single article: 'url', 'text', and 'title'.

    Returns:
        text_dict: Dictionary where each item is a string of the text of an article, starting with the title.
    """
    if type(header) == int:
        header = str(header) 
    else :
        header = rf"[{''.join([str(h) for h in range(header[0], header[-1]+1)])}]"
    print(rf'header: {header}')
    article_regex_str = article_regex_str.replace('\d', header)
    abs_regex_str = abs_regex_str.replace('\d', header)
    article_regex = rf'{article_regex_str}'
    abs_regex = rf'{abs_regex_str}'
    print(f'Regex patterns: \n\t{article_regex}\n\t{abs_regex}')
    text_dict = dict()
    display_dict = dict()
    if (type(to_display) == int) or (type(to_display) == float):
        to_display = [to_display] 
    for article_key in article_dict:
        if (article_key +1) - (article_key +1) //1 == 0: # if integer
            print(f'Journal: {article_dict[article_key]["journal"]} {article_key}')
        trimmed_text, display = trim_text(article_dict[article_key]['text'], article_regex, abs_regex)
        text_dict[article_key] = {
            'title': article_dict[article_key]['title'],
            'body': f"{article_dict[article_key]['title']}\n\n{trimmed_text['body']}",
            'abstract': trimmed_text['abstract'],
        }
        if (to_display == 'all') or (to_display == None) or (article_key in to_display):
            display_dict[article_key] = {
                'abstract': display['abs_display'],
                'body': display['article_display']
            }
    print(f'text_dict keys: {[key for key in text_dict.keys()]}')
    return text_dict, display_dict

def display_html(display_dict, type='abstract'):
    """
    Display the HTML from the dictionary of HTML displays.
    """
    print()
    for text in display_dict:
        print('************************************* Start *************************************')
        display.display(display_dict[text][type])


# root_display_dict = dict()
# text_dict = dict()
iteration = 1
# regex = r'(.*)'
# text_dict[iteration], display_dict = text_dict_from_web(
#     partial_article_dict[text_id], to_display='all', header=(2,4), article_regex_str=regex, abs_regex_str=regex)

text_dict[iteration], display_dict = text_dict_from_web(
    partial_article_dict[text_id], to_display='all', header=(2,4))

root_display_dict[iteration] = display_dict
display_html(display_dict, type='abstract') 


header: [234]
Regex patterns: 
	.*<h[234].*?>Abstract</h[234]>.*(?:Introduction.*)?(<h[234].*?>Introduction</h[234]>.*References)<.*
	.*(<h[234].*?>Abstract</h[234]>.*(?:Introduction.*)?)<h[234].*?>Introduction</h[234]>.*References<.*
Journal: Journal of Medical Internet Research 2.0
intro_header: Introduction
intro_header: Introduction
intro_header: Introduction
intro_header: Introduction


intro_header: Introduction
intro_header: Introduction
text_dict keys: [2.0, 2.01, 2.02, 2.03, 2.04, 2.05]

************************************* Start *************************************


************************************* Start *************************************


************************************* Start *************************************


************************************* Start *************************************


************************************* Start *************************************


************************************* Start *************************************


## text_id = 1 Try with more articles

In [9]:
text_id = 1
partial_article_dict[text_id] = create_partial_article_dict(article_dict[0], n_articles=30, journals=1)

Keys for article_dict: [1.0, 1.01, 1.02, 1.03, 1.04, 1.05, 1.06, 1.07, 1.08, 1.09, 1.1, 1.11, 1.12, 1.13, 1.14, 1.15, 1.16, 1.17, 1.18, 1.19, 1.2, 1.21, 1.22, 1.23, 1.24, 1.25, 1.26, 1.27, 1.28, 1.29]
Journals:
	BMJ Open


## Iteration 1.1

In [32]:
from IPython import display

def trim_text(text, article_regex=None, abs_regex=None):
    if article_regex==None:
        article_regex = '.*<h2>Abstract</h2>.*(?:Introduction.*)?(<h2.*?>Introduction</h2>.*References)<.*' 
        abs_regex = '.*(<h2>Abstract</h2>.*(?:Introduction.*)?)<h2.*?>Introduction</h2>.*References<.*' 

    intro_header = '>Introduction<'
    
    print(f'\tintro_header: {intro_header}')
    if intro_header in text:
        try:
            body = re.search(article_regex, text, re.DOTALL).group(1)
            abstract = re.search(abs_regex, text, re.DOTALL).group(1)
        except Exception as error: 
            exc_type, exc_obj, tb = sys.exc_info()
            file = tb.tb_frame
            lineno = tb.tb_lineno
            filename = file.f_code.co_filename
            print(f'\tAn error occurred on line {lineno} in {filename}: {error}')    
            print('\t\tUnable to parse article text')
            body = text 
            abstract = text 
        try:
            article_display = display.HTML(body)
            abs_display = display.HTML(abstract)
        except Exception as error: 
            exc_type, exc_obj, tb = sys.exc_info()
            file = tb.tb_frame
            lineno = tb.tb_lineno
            filename = file.f_code.co_filename
            print(f'\tAn error occurred on line {lineno} in {filename}: {error}')    
            print('\t\tUnable to create HTML display')
            article_display = f'<p>{body}</p>'
            abs_display = f'<p>{abstract}</p>'
    else:
        print('No introduction header found')
        body = text 
        abstract = text 
        article_display = f'<p>{body}</p>'
        abs_display = f'<p>{abstract}</p>'
    processed_article = {
        'abstract': abstract,
        'body': body,
    }
    display_dict = {
        'article_display': article_display,
        'abs_display': abs_display
    }
    return processed_article, display_dict

def text_dict_from_web(article_dict, header=(2,4), to_display=0.01, verbose=False,
        article_regex_str=r'.*<h\d.*?>Abstract</h\d>.*(?:Introduction.*)?(<h\d.*?>Introduction</h\d>.*References)<.*',
        abs_regex_str=r'.*(<h\d.*?>Abstract</h\d>.*(?:Introduction.*)?)<h\d.*?>Introduction</h\d>.*References<.*'
        ):
    """
    Create a text dictionary from a dictionary containing web-scraped articles.

    Parameters:
        article_dict (dict): Values of each dictionary item are a dictionary representing the data from a 
            single article: 'url', 'text', and 'title'.

    Returns:
        text_dict: Dictionary where each item is a string of the text of an article, starting with the title.
    """
    if type(header) == int:
        header = str(header) 
    else :
        header = rf"[{''.join([str(h) for h in range(header[0], header[-1]+1)])}]"
    print(rf'header: {header}')
    article_regex_str = article_regex_str.replace('\d', header)
    abs_regex_str = abs_regex_str.replace('\d', header)
    article_regex = rf'{article_regex_str}'
    abs_regex = rf'{abs_regex_str}'
    print(f'Regex patterns: \n\t{article_regex}\n\t{abs_regex}')
    text_dict = dict()
    display_dict = dict()
    if (type(to_display) == int) or (type(to_display) == float):
        to_display = [to_display] 
    for article_key in article_dict:
        if (verbose == True) or  ((article_key +1) - (article_key +1) //1 == 0): # if integer
            print(f'Journal: {article_dict[article_key]["journal"]} {article_key}')
        trimmed_text, display = trim_text(article_dict[article_key]['text'], article_regex, abs_regex)
        text_dict[article_key] = {
            'title': article_dict[article_key]['title'],
            'body': f"{article_dict[article_key]['title']}\n\n{trimmed_text['body']}",
            'abstract': trimmed_text['abstract'],
        }
        if (to_display == 'all') or (to_display == None) or (article_key in to_display):
            display_dict[article_key] = {
                'abstract': display['abs_display'],
                'body': display['article_display']
            }
    print(f'text_dict keys: {[key for key in text_dict.keys()]}')
    return text_dict, display_dict

def display_html(display_dict, type='abstract'):
    """
    Display the HTML from the dictionary of HTML displays.
    """
    print()
    for text in display_dict:
        print('************************************* Start *************************************')
        display.display(display_dict[text][type])


# root_display_dict = dict()
# text_dict = dict()
iteration = 1.1
# regex = r'(.*)'
# text_dict[iteration], display_dict = text_dict_from_web(
#     partial_article_dict[text_id], to_display='all', header=(2,4), article_regex_str=regex, abs_regex_str=regex)

text_dict[iteration], display_dict = text_dict_from_web(
    partial_article_dict[text_id], to_display='all', header=(2,4), verbose=True)

root_display_dict[iteration] = display_dict
display_html(display_dict, type='abstract')


header: [234]
Regex patterns: 
	.*<h[234].*?>Abstract</h[234]>.*(?:Introduction.*)?(<h[234].*?>Introduction</h[234]>.*References)<.*
	.*(<h[234].*?>Abstract</h[234]>.*(?:Introduction.*)?)<h[234].*?>Introduction</h[234]>.*References<.*
Journal: BMJ Open 1.0
	intro_header: >Introduction<
Journal: BMJ Open 1.01
	intro_header: >Introduction<
Journal: BMJ Open 1.02
	intro_header: >Introduction<
Journal: BMJ Open 1.03
	intro_header: >Introduction<
Journal: BMJ Open 1.04
	intro_header: >Introduction<
Journal: BMJ Open 1.05
	intro_header: >Introduction<
Journal: BMJ Open 1.06
	intro_header: >Introduction<
Journal: BMJ Open 1.07
	intro_header: >Introduction<
	An error occurred on line 13 in C:\Users\silvh\AppData\Local\Temp\ipykernel_8516\618142999.py: 'NoneType' object has no attribute 'group'
		Unable to parse article text
Journal: BMJ Open 1.08
	intro_header: >Introduction<
Journal: BMJ Open 1.09
	intro_header: >Introduction<
Journal: BMJ Open 1.1
	intro_header: >Introduction<
	An error occu

************************************* Start *************************************


************************************* Start *************************************


************************************* Start *************************************


************************************* Start *************************************


...

************************************* Start *************************************


************************************* Start *************************************


************************************* Start *************************************


************************************* Start *************************************


'<p>\n<p class="logo-bmj-journals"><a href="http://journals.bmj.com/" title="BMJ Journals"><img src="/sites/default/themes/bmjj/img/logos/logo-bmj-journals.svg" title="BMJ Journals" alt="BMJ Journals"></a></p>\n<h3>Log in using your username and password</h3>\n<h2 class="pane-title"><span class="icon-search"></span></h2>\n<h2 class="pane-title"><span class="icon-menu"></span></h2>\n<h2 class="pane-title">Main menu</h2>\n<h3>Log in using your username and password</h3>\n<h2 class="element-invisible">You are here</h2>\n<p><span class="icon-article-text"></span>Article Text</p>\n<p><a class="article-pdf-download" href="/content/bmjopen/13/6/e071670.full.pdf" target="new"><img alt="Download PDF" src="/sites/default/themes/bmjj/img/icon-pdf.png"><strong>PDF</strong></a></p>\n<p><a class="article-pdf-download" href="/content/bmjopen/13/6/e071670.full.pdf?with-ds=yes" target="new"><img alt="Download PDF + Supplemental Data" src="/sites/default/themes/bmjj/img/icon-pdf.png"><strong>PDF +<br>\n

In [33]:
'Introduction' in partial_article_dict[text_id][1.07]['text']

True

In [29]:
'Reference' in partial_article_dict[text_id][1.02]['text']

True

In [30]:
'Abstract' in partial_article_dict[text_id][1.02]['text']

True

In [31]:
article_regex = '.*<h[234].*?>Abstract</h[234]>.*(?:Introduction.*)?(<h[234].*?>Introduction</h[234]>.*References)<.*'

re.search(article_regex, partial_article_dict[text_id][1.02]['text'], re.DOTALL).group(1)

'<h2 class="">Introduction</h2>\n<p id="p-14">As COVID-19 spread within countries, vulnerable and marginalised populations such as specific ethnic minorities and migrant or refugee groups have been unduly affected. Figures released by the Australian Bureau of Statistics in early 2022 indicated that the COVID-19-related death rate for people born overseas was 6.8 per 100 000, compared with 2.3 for those born in Australia.<a id="xref-ref-1-1" class="xref-bibr" href="#ref-1">1</a> Further, the mortality rate for people born in the Middle East (29.3 deaths per 100 000) was 10 times than that of those born in Australia. International data also support that compared with the white population majority, there has been a much greater risk of infection and adverse outcomes from COVID-19 among black, Asian, and minority ethnic groups, black Americans, Hispanics, Latinos, other people of colour and Indigenous groups.<a id="xref-ref-2-1" class="xref-bibr" href="#ref-2">2</a></p>\n<p id="p-15">There

## 1.2 Parse using either 'Introduction' or 'Background' headers

In [11]:
from IPython import display

def trim_text(text, article_regex=None, abs_regex=None):
    if article_regex==None:
        article_regex = '.*<h2>Abstract</h2>.*(?:Introduction.*)?(<h2.*?>Introduction</h2>.*References)<.*' 
        abs_regex = '.*(<h2>Abstract</h2>.*(?:Introduction.*)?)<h2.*?>Introduction</h2>.*References<.*' 
    try:
        body = re.search(article_regex, text, re.DOTALL).group(1)
        abstract = re.search(abs_regex, text, re.DOTALL).group(1)
    except Exception as error: 
        exc_type, exc_obj, tb = sys.exc_info()
        file = tb.tb_frame
        lineno = tb.tb_lineno
        filename = file.f_code.co_filename
        print(f'\tAn error occurred on line {lineno} in {filename}: {error}')    
        print('\t\tUnable to parse article text')
        body = text 
        abstract = text 
    try:
        article_display = display.HTML(body)
        abs_display = display.HTML(abstract)
    except Exception as error: 
        exc_type, exc_obj, tb = sys.exc_info()
        file = tb.tb_frame
        lineno = tb.tb_lineno
        filename = file.f_code.co_filename
        print(f'\tAn error occurred on line {lineno} in {filename}: {error}')    
        print('\t\tUnable to create HTML display')
        article_display = f'<p>{body}</p>'
        abs_display = f'<p>{abstract}</p>'
    processed_article = {
        'abstract': abstract,
        'body': body,
    }
    display_dict = {
        'article_display': article_display,
        'abs_display': abs_display
    }
    return processed_article, display_dict

def text_dict_from_web(article_dict, header=(2,4), to_display=0.01, verbose=False,
        article_regex_str=r'.*<h\d.*?>Abstract</h\d>.*(?:(?:Introduction|Background).*)?(<h\d.*?>(?:Introduction|Background)</h\d>.*References)<.*',
        abs_regex_str=r'.*(<h\d.*?>Abstract</h\d>.*(?:(?:Introduction|Background).*)?)<h\d.*?>(?:Introduction|Background)</h\d>.*References<.*'
        ):
    """
    Create a text dictionary from a dictionary containing web-scraped articles.

    Parameters:
        article_dict (dict): Values of each dictionary item are a dictionary representing the data from a 
            single article: 'url', 'text', and 'title'.

    Returns:
        text_dict: Dictionary where each item is a string of the text of an article, starting with the title.
    """
    if type(header) == int:
        header = str(header) 
    else :
        header = rf"[{''.join([str(h) for h in range(header[0], header[-1]+1)])}]"
    print(rf'header: {header}')
    article_regex_str = article_regex_str.replace('\d', header)
    abs_regex_str = abs_regex_str.replace('\d', header)
    article_regex = rf'{article_regex_str}'
    abs_regex = rf'{abs_regex_str}'
    print(f'Regex patterns: \n\t{article_regex}\n\t{abs_regex}')
    text_dict = dict()
    display_dict = dict()
    if (type(to_display) == int) or (type(to_display) == float):
        to_display = [to_display] 
    for article_key in article_dict:
        if (verbose == True) or  ((article_key +1) - (article_key +1) //1 == 0): # if integer
            print(f'Journal: {article_dict[article_key]["journal"]} {article_key}')
        trimmed_text, display = trim_text(article_dict[article_key]['text'], article_regex, abs_regex)
        text_dict[article_key] = {
            'title': article_dict[article_key]['title'],
            'body': f"{article_dict[article_key]['title']}\n\n{trimmed_text['body']}",
            'abstract': trimmed_text['abstract'],
        }
        if (to_display == 'all') or (to_display == None) or (article_key in to_display):
            display_dict[article_key] = {
                'abstract': display['abs_display'],
                'body': display['article_display']
            }
    print(f'text_dict keys: {[key for key in text_dict.keys()]}')
    return text_dict, display_dict

def display_html(display_dict, type='abstract'):
    """
    Display the HTML from the dictionary of HTML displays.
    """
    print()
    for text in display_dict:
        print('************************************* Start *************************************')
        display.display(display_dict[text][type])


In [41]:


# root_display_dict = dict()
# text_dict = dict()
iteration = 1.21
text_id = 1

# regex = r'(.*)'
# text_dict[iteration], display_dict = text_dict_from_web(
#     partial_article_dict[text_id], to_display='all', header=(2,4), article_regex_str=regex, abs_regex_str=regex)

text_dict[iteration], display_dict = text_dict_from_web(
    partial_article_dict[text_id], to_display='all', header=(2,4), verbose=True)

root_display_dict[iteration] = display_dict
display_html(display_dict, type='abstract')


header: [234]
Regex patterns: 
	.*<h[234].*?>Abstract</h[234]>.*(?:(?:Introduction|Background).*)?(<h[234].*?>(?:Introduction|Background)</h[234]>.*References)<.*
	.*(<h[234].*?>Abstract</h[234]>.*(?:(?:Introduction|Background).*)?)<h[234].*?>(?:Introduction|Background)</h[234]>.*References<.*
Journal: BMJ Open 1.0
Journal: BMJ Open 1.01


Journal: BMJ Open 1.02
Journal: BMJ Open 1.03
Journal: BMJ Open 1.04
Journal: BMJ Open 1.05
Journal: BMJ Open 1.06
Journal: BMJ Open 1.07
Journal: BMJ Open 1.08
Journal: BMJ Open 1.09
Journal: BMJ Open 1.1
Journal: BMJ Open 1.11
Journal: BMJ Open 1.12
Journal: BMJ Open 1.13
Journal: BMJ Open 1.14
Journal: BMJ Open 1.15
Journal: BMJ Open 1.16
Journal: BMJ Open 1.17
Journal: BMJ Open 1.18
Journal: BMJ Open 1.19
Journal: BMJ Open 1.2
Journal: BMJ Open 1.21
Journal: BMJ Open 1.22
Journal: BMJ Open 1.23
Journal: BMJ Open 1.24
Journal: BMJ Open 1.25
Journal: BMJ Open 1.26
Journal: BMJ Open 1.27
Journal: BMJ Open 1.28
Journal: BMJ Open 1.29
text_dict keys: [1.0, 1.01, 1.02, 1.03, 1.04, 1.05, 1.06, 1.07, 1.08, 1.09, 1.1, 1.11, 1.12, 1.13, 1.14, 1.15, 1.16, 1.17, 1.18, 1.19, 1.2, 1.21, 1.22, 1.23, 1.24, 1.25, 1.26, 1.27, 1.28, 1.29]

************************************* Start *************************************


************************************* Start *************************************


************************************* Start *************************************


************************************* Start *************************************


...

************************************* Start *************************************


************************************* Start *************************************


************************************* Start *************************************


************************************* Start *************************************


************************************* Start *************************************


In [43]:
display_dict[1.07]['abstract']

In [44]:
display_dict[1.07]['body']

## text_id = 2, iteration 1.22 test with more journal articles

In [6]:
text_id = 2
partial_article_dict[text_id] = create_partial_article_dict(article_dict[0], n_articles=30, journals='all')

Keys for article_dict: [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 1.0, 1.01, 1.02, 1.03, 1.04, 1.05, 1.06, 1.07, 1.08, 1.09, 1.1, 1.11, 1.12, 1.13, 1.14, 1.15, 1.16, 1.17, 1.18, 1.19, 1.2, 1.21, 1.22, 1.23, 1.24, 1.25, 1.26, 1.27, 1.28, 1.29, 2.0, 2.01, 2.02, 2.03, 2.04, 2.05, 2.06, 2.07, 2.08, 2.09, 3.0, 3.01, 3.02, 3.03, 3.04, 3.05, 3.06, 3.07, 3.08, 3.09, 3.1, 3.11, 3.12, 3.13, 3.14, 3.15, 3.16, 3.17, 3.18, 3.19, 3.2, 3.21, 3.22, 3.23, 3.24, 3.25, 3.26, 3.27, 3.28, 3.29]
Journals:
	Journal of Medical Internet Research
	PLOS One
	BMJ Open
	PLOS Medicine


In [ ]:
iteration = 1.22

# regex = r'(.*)'
# text_dict[iteration], display_dict = text_dict_from_web(
#     partial_article_dict[text_id], to_display='all', header=(2,4), article_regex_str=regex, abs_regex_str=regex)

text_dict[iteration], display_dict = text_dict_from_web(
    partial_article_dict[text_id], to_display='all', header=(2,4), verbose=True)

root_display_dict[iteration] = display_dict
display_html(display_dict, type='abstract')

header: [234]
Regex patterns: 
	.*<h[234].*?>Abstract</h[234]>.*(?:(?:Introduction|Background).*)?(<h[234].*?>(?:Introduction|Background)</h[234]>.*References)<.*
	.*(<h[234].*?>Abstract</h[234]>.*(?:(?:Introduction|Background).*)?)<h[234].*?>(?:Introduction|Background)</h[234]>.*References<.*
Journal: BMJ Open 1.0
Journal: BMJ Open 1.01
Journal: BMJ Open 1.02
Journal: BMJ Open 1.03
Journal: BMJ Open 1.04
Journal: BMJ Open 1.05
Journal: BMJ Open 1.06
Journal: BMJ Open 1.07
Journal: BMJ Open 1.08
Journal: BMJ Open 1.09
Journal: BMJ Open 1.1
Journal: BMJ Open 1.11
Journal: BMJ Open 1.12
Journal: BMJ Open 1.13
Journal: BMJ Open 1.14
Journal: BMJ Open 1.15
Journal: BMJ Open 1.16
Journal: BMJ Open 1.17
Journal: BMJ Open 1.18
Journal: BMJ Open 1.19
Journal: BMJ Open 1.2
Journal: BMJ Open 1.21
Journal: BMJ Open 1.22
Journal: BMJ Open 1.23
Journal: BMJ Open 1.24
Journal: BMJ Open 1.25
Journal: BMJ Open 1.26
Journal: BMJ Open 1.27
Journal: BMJ Open 1.28
Journal: BMJ Open 1.29
Journal: PLOS Medi

Issues with some articles

PLOS Medicine
* Article 3.2: Not a journal article, but a correction
* 3.23: Editorial
* 3.29: Perspective
Issues with PLOS One
* 0.03: '1. Introduction'
* 0.16: '1 Introduction'

PLOS Medicine journals have `<p class="type-correction" id="artType">Correction</p>` to indicate article type


#### parse article type

In [30]:
article_type_regex = r'id="artType">(.+?)<.*'

re.search(article_type_regex, partial_article_dict[text_id][3.23]['text'], re.DOTALL).group(1)

'Editorial'

#### What is the issue with 0.16

In [12]:
article_regex = '.*<h[234].*?>Abstract</h[234]>.*(?:Introduction.*)?(<h[234].*?>1 Introduction</h[234]>.*References)<.*'

re.search(article_regex, partial_article_dict[text_id][0.16]['text'], re.DOTALL).group(1)

'<h2>1 Introduction</h2>\n<p>Nowadays, the utilization of blockchain technology is in increase day after day [<a href="#pone.0286667.ref001" class="ref-tip">1</a>]. Person use of blockchain technology has also remarkably increased since 2016. The statistics published by several sources in 2020 show that there were more than 40 million blockchain wallets this year in comparison to 10 million blockchain wallets in 2016. The blockchain is integrated into several areas. Indeed, cryptocurrencies are used in a large way the blockchain technology. The security of cryptocurrencies in blockchain technology is studied giving state-of-art challenges and future prospects in [<a href="#pone.0286667.ref002" class="ref-tip">2</a>]. Various category is presented in the latter work to show the importance of the blockchain applied to cryptocurrencies.</p>\n<p>In the face of multiple transactions and a huge number of transactions, good management of these transactions optimize the use of the blocks to re

In [16]:
article_regex = '.*<h[234].*?>Abstract</h[234]>.*(?:Introduction.*)?(<h[234].*?>\d?.?\s?Introduction</h[234]>.*References)<.*'

re.search(article_regex, partial_article_dict[text_id][0.16]['text'], re.DOTALL).group(1)

'<h2>1 Introduction</h2>\n<p>Nowadays, the utilization of blockchain technology is in increase day after day [<a href="#pone.0286667.ref001" class="ref-tip">1</a>]. Person use of blockchain technology has also remarkably increased since 2016. The statistics published by several sources in 2020 show that there were more than 40 million blockchain wallets this year in comparison to 10 million blockchain wallets in 2016. The blockchain is integrated into several areas. Indeed, cryptocurrencies are used in a large way the blockchain technology. The security of cryptocurrencies in blockchain technology is studied giving state-of-art challenges and future prospects in [<a href="#pone.0286667.ref002" class="ref-tip">2</a>]. Various category is presented in the latter work to show the importance of the blockchain applied to cryptocurrencies.</p>\n<p>In the face of multiple transactions and a huge number of transactions, good management of these transactions optimize the use of the blocks to re

#### What is the issue with 0.03

In [17]:
article_regex = '.*<h[234].*?>Abstract</h[234]>.*(?:Introduction.*)?(<h[234].*?>\d?.?\s?Introduction</h[234]>.*References)<.*'

re.search(article_regex, partial_article_dict[text_id][0.03]['text'], re.DOTALL).group(1)

'<h2>1. Introduction</h2>\n<p>Photo catalysis is the hottest field having large applications of nanomaterials as photo catalysts especially in water treatment and hydrogen generation from water splitting [<a href="#pone.0287424.ref001" class="ref-tip">1</a>]. In green hydrogen production, photo catalysis can be used to split water molecules into hydrogen and oxygen, providing a clean and renewable source of energy. The photons from the light excite the electrons in a semiconductor photocatalyst, allowing them and the corresponding formed holes to participate in the reaction. As a result, water molecules are broken down into hydrogen and oxygen. This process is considered green because it does not generate greenhouse gases or other pollutants [<a href="#pone.0287424.ref002" class="ref-tip">2</a>, <a href="#pone.0287424.ref003" class="ref-tip">3</a>].</p>\n<p>In the degradation of organic pollutants, photo catalysis is used to break down organic pollutants in the environment [<a href="#p

# text_id = 3 PLOS Medicine

In [41]:
text_id = 3
partial_article_dict[text_id] = create_partial_article_dict(article_dict[0], n_articles=30, journals=3)

Keys for article_dict: [3.0, 3.01, 3.02, 3.03, 3.04, 3.05, 3.06, 3.07, 3.08, 3.09, 3.1, 3.11, 3.12, 3.13, 3.14, 3.15, 3.16, 3.17, 3.18, 3.19, 3.2, 3.21, 3.22, 3.23, 3.24, 3.25, 3.26, 3.27, 3.28, 3.29]
Journals:
	PLOS Medicine


# Iteration 2 remove articles that are not original research

In [39]:

types_to_exclude = ['Editorial', 'Correction', 'Retraction', 'Expression of Concern', 'Perspective']
types_to_exclude += [type.upper() for type in types_to_exclude]
types_to_exclude

['Editorial',
 'Correction',
 'Retraction',
 'Expression of Concern',
 'Perspective',
 'EDITORIAL',
 'CORRECTION',
 'RETRACTION',
 'EXPRESSION OF CONCERN',
 'PERSPECTIVE']

In [48]:
from IPython import display

def trim_text(text, article_regex=None, abs_regex=None):
    if article_regex==None:
        article_regex = '.*<h2>Abstract</h2>.*(?:(?:Introduction|Background).*)?(<h2.*?>\d?.?\s?(?:Introduction|Background)</h2>.*References)<.*' 
        abs_regex = '.*(<h2>Abstract</h2>.*(?:(?:Introduction|Background).*)?)<h2.*?>\d?.?\s?(?:Introduction|Background)</h2>.*References<.*' 
    try:
        body = re.search(article_regex, text, re.DOTALL).group(1)
        abstract = re.search(abs_regex, text, re.DOTALL).group(1)
    except Exception as error: 
        exc_type, exc_obj, tb = sys.exc_info()
        file = tb.tb_frame
        lineno = tb.tb_lineno
        filename = file.f_code.co_filename
        print(f'\tAn error occurred on line {lineno} in {filename}: {error}')    
        print('\t\tUnable to parse article text')
        body = text 
        abstract = text 
    try:
        article_display = display.HTML(body)
        abs_display = display.HTML(abstract)
    except Exception as error: 
        exc_type, exc_obj, tb = sys.exc_info()
        file = tb.tb_frame
        lineno = tb.tb_lineno
        filename = file.f_code.co_filename
        print(f'\tAn error occurred on line {lineno} in {filename}: {error}')    
        print('\t\tUnable to create HTML display')
        article_display = f'<p>{body}</p>'
        abs_display = f'<p>{abstract}</p>'
    processed_article = {
        'abstract': abstract,
        'body': body,
    }
    display_dict = {
        'article_display': article_display,
        'abs_display': abs_display
    }
    return processed_article, display_dict

def text_dict_from_web(article_dict, header=(2,4), to_display=0.01, verbose=False,
        article_regex_str=r'.*<h\d.*?>Abstract</h\d>.*(?:(?:Introduction|Background).*)?(<h\d.*?>\d?.?\s?(?:Introduction|Background)</h\d>.*References)<.*',
        abs_regex_str=r'.*(<h\d.*?>Abstract</h\d>.*(?:(?:Introduction|Background).*)?)<h\d.*?>\d?.?\s?(?:Introduction|Background)</h\d>.*References<.*'
        ):
    """
    Create a text dictionary from a dictionary containing web-scraped articles.

    Parameters:
        article_dict (dict): Values of each dictionary item are a dictionary representing the data from a 
            single article: 'url', 'text', and 'title'.

    Returns:
        text_dict: Dictionary where each item is a string of the text of an article, starting with the title.
    """
    if type(header) == int:
        header = str(header) 
    else :
        header = rf"[{''.join([str(h) for h in range(header[0], header[-1]+1)])}]"
    print(rf'header: {header}')
    article_regex_str = article_regex_str.replace('\d', header)
    abs_regex_str = abs_regex_str.replace('\d', header)
    article_regex = rf'{article_regex_str}'
    abs_regex = rf'{abs_regex_str}'
    print(f'Regex patterns: \n\t{article_regex}\n\t{abs_regex}')
    text_dict = dict()
    display_dict = dict()
    if (type(to_display) == int) or (type(to_display) == float):
        to_display = [to_display] 
    for article_key in article_dict:
        journal = article_dict[article_key]["journal"]
        text = article_dict[article_key]['text']
        if 'PLOS' in journal:
            article_type_regex = r'id="artType">(.+?)<.*'
            article_type = re.search(article_type_regex, text, re.DOTALL).group(1)
            types_to_exclude = ['Editorial', 'Correction', 'Perspective', 'Retraction']
            types_to_exclude += [type.lower() for type in types_to_exclude]
            types_to_exclude += [type.upper() for type in types_to_exclude]
        else:
            article_type = 'Research Article'
        if (verbose == True) or  ((article_key +1) - (article_key +1) //1 == 0): # if integer
            print(f'Journal: {journal} {article_key}')
            # print(f'\tArticle type: {article_type}')
        if (article_type == 'Research Article') or (article_type not in types_to_exclude):
            trimmed_text, display = trim_text(text, article_regex, abs_regex)
            text_dict[article_key] = {
                'title': article_dict[article_key]['title'],
                'body': f"{article_dict[article_key]['title']}\n\n{trimmed_text['body']}",
                'abstract': trimmed_text['abstract'],
            }
            if (to_display == 'all') or (to_display == None) or (article_key in to_display):
                display_dict[article_key] = {
                    'abstract': display['abs_display'],
                    'body': display['article_display']
                }
        else:
            print(f'\tArticle type "{article_type}" excluded')
    print(f'text_dict keys: {[key for key in text_dict.keys()]}')
    return text_dict, display_dict

def display_html(display_dict, type='abstract'):
    """
    Display the HTML from the dictionary of HTML displays.
    """
    print()
    for text in display_dict:
        print('************************************* Start *************************************')
        display.display(display_dict[text][type])


# root_display_dict = dict()
# text_dict = dict()
iteration = 2
text_id = 3

# regex = r'(.*)'
# text_dict[iteration], display_dict = text_dict_from_web(
#     partial_article_dict[text_id], to_display='all', header=(2,4), article_regex_str=regex, abs_regex_str=regex)

text_dict[iteration], display_dict = text_dict_from_web(
    partial_article_dict[text_id], to_display='all', header=(2,4), verbose=True)

root_display_dict[iteration] = display_dict
display_html(display_dict, type='abstract')



header: [234]
Regex patterns: 
	.*<h[234].*?>Abstract</h[234]>.*(?:(?:Introduction|Background).*)?(<h[234].*?>[234]?.?\s?(?:Introduction|Background)</h[234]>.*References)<.*
	.*(<h[234].*?>Abstract</h[234]>.*(?:(?:Introduction|Background).*)?)<h[234].*?>[234]?.?\s?(?:Introduction|Background)</h[234]>.*References<.*
Journal: PLOS Medicine 3.0
Journal: PLOS Medicine 3.01
Journal: PLOS Medicine 3.02
Journal: PLOS Medicine 3.03
Journal: PLOS Medicine 3.04
Journal: PLOS Medicine 3.05
Journal: PLOS Medicine 3.06
Journal: PLOS Medicine 3.07
Journal: PLOS Medicine 3.08
Journal: PLOS Medicine 3.09
Journal: PLOS Medicine 3.1
Journal: PLOS Medicine 3.11
Journal: PLOS Medicine 3.12
Journal: PLOS Medicine 3.13
Journal: PLOS Medicine 3.14
Journal: PLOS Medicine 3.15
Journal: PLOS Medicine 3.16
Journal: PLOS Medicine 3.17
Journal: PLOS Medicine 3.18
Journal: PLOS Medicine 3.19
Journal: PLOS Medicine 3.2
	Article type "Correction" excluded
Journal: PLOS Medicine 3.21
Journal: PLOS Medicine 3.22
Journa

************************************* Start *************************************


************************************* Start *************************************


************************************* Start *************************************


************************************* Start *************************************


...

************************************* Start *************************************


************************************* Start *************************************


************************************* Start *************************************


************************************* Start *************************************


************************************* Start *************************************


## 2.1 test on all articles

In [ ]:
iteration = 2.1
text_id = 2

# regex = r'(.*)'
# text_dict[iteration], display_dict = text_dict_from_web(
#     partial_article_dict[text_id], to_display='all', header=(2,4), article_regex_str=regex, abs_regex_str=regex)

text_dict[iteration], display_dict = text_dict_from_web(
    partial_article_dict[text_id], to_display='all', header=(2,4), verbose=True)

root_display_dict[iteration] = display_dict
display_html(display_dict, type='abstract')

header: [234]
Regex patterns: 
	.*<h[234].*?>Abstract</h[234]>.*(?:(?:Introduction|Background).*)?(<h[234].*?>[234]?.?\s?(?:Introduction|Background)</h[234]>.*References)<.*
	.*(<h[234].*?>Abstract</h[234]>.*(?:(?:Introduction|Background).*)?)<h[234].*?>[234]?.?\s?(?:Introduction|Background)</h[234]>.*References<.*
Journal: BMJ Open 1.0
Journal: BMJ Open 1.01
Journal: BMJ Open 1.02
Journal: BMJ Open 1.03
Journal: BMJ Open 1.04
Journal: BMJ Open 1.05
Journal: BMJ Open 1.06
Journal: BMJ Open 1.07
Journal: BMJ Open 1.08
Journal: BMJ Open 1.09
Journal: BMJ Open 1.1
Journal: BMJ Open 1.11
Journal: BMJ Open 1.12
Journal: BMJ Open 1.13
Journal: BMJ Open 1.14
Journal: BMJ Open 1.15
Journal: BMJ Open 1.16
Journal: BMJ Open 1.17
Journal: BMJ Open 1.18
Journal: BMJ Open 1.19
Journal: BMJ Open 1.2
Journal: BMJ Open 1.21
Journal: BMJ Open 1.22
Journal: BMJ Open 1.23
Journal: BMJ Open 1.24
Journal: BMJ Open 1.25
Journal: BMJ Open 1.26
Journal: BMJ Open 1.27
Journal: BMJ Open 1.28
Journal: BMJ Open 1

## ** 2.2 best version to continue with

In [7]:
from IPython import display

def trim_text(text, article_regex=None, abs_regex=None):
    if article_regex==None:
        article_regex = '.*<h2>Abstract</h2>.*(?:(?:Introduction|Background).*)?(<h2.*?>\d?.?\s?(?:Introduction|Background)</h2>.*References)<.*' 
        abs_regex = '.*(<h2>Abstract</h2>.*(?:(?:Introduction|Background).*)?)<h2.*?>\d?.?\s?(?:Introduction|Background)</h2>.*References<.*' 
    try:
        body = re.search(article_regex, text, re.DOTALL).group(1)
        abstract = re.search(abs_regex, text, re.DOTALL).group(1)
    except Exception as error: 
        exc_type, exc_obj, tb = sys.exc_info()
        file = tb.tb_frame
        lineno = tb.tb_lineno
        filename = file.f_code.co_filename
        print(f'\tAn error occurred on line {lineno} in {filename}: {error}')    
        print('\t\tUnable to parse article text')
        body = text 
        abstract = text 
    try:
        article_display = display.HTML(body)
        abs_display = display.HTML(abstract)
    except Exception as error: 
        exc_type, exc_obj, tb = sys.exc_info()
        file = tb.tb_frame
        lineno = tb.tb_lineno
        filename = file.f_code.co_filename
        print(f'\tAn error occurred on line {lineno} in {filename}: {error}')    
        print('\t\tUnable to create HTML display')
        article_display = f'<p>{body}</p>'
        abs_display = f'<p>{abstract}</p>'
    processed_article = {
        'abstract': abstract,
        'body': body,
    }
    display_dict = {
        'article_display': article_display,
        'abs_display': abs_display
    }
    return processed_article, display_dict

def text_dict_from_web(article_dict, header=(2,4), to_display=0.01, verbose=False,
        article_regex_str=r'.*<h\d.*?>Abstract</h\d>.*(?:(?:Introduction|Background).*)?(<h\d.*?>1?.?\s?(?:Introduction|Background)</h\d>.*References)<.*',
        abs_regex_str=r'.*(<h\d.*?>Abstract</h\d>.*(?:(?:Introduction|Background).*)?)<h\d.*?>1?.?\s?(?:Introduction|Background)</h\d>.*References<.*'
        ):
    """
    Create a text dictionary from a dictionary containing web-scraped articles.

    Parameters:
        article_dict (dict): Values of each dictionary item are a dictionary representing the data from a 
            single article: 'url', 'text', and 'title'.

    Returns:
        text_dict: Dictionary where each item is a string of the text of an article, starting with the title.
    """
    if type(header) == int:
        header = str(header) 
    else :
        header = rf"[{''.join([str(h) for h in range(header[0], header[-1]+1)])}]"
    print(rf'header: {header}')
    article_regex_str = article_regex_str.replace('\d', header)
    abs_regex_str = abs_regex_str.replace('\d', header)
    article_regex = rf'{article_regex_str}'
    abs_regex = rf'{abs_regex_str}'
    print(f'Regex patterns: \n\t{article_regex}\n\t{abs_regex}')
    text_dict = dict()
    display_dict = dict()
    if (type(to_display) == int) or (type(to_display) == float):
        to_display = [to_display] 
    for article_key in article_dict:
        journal = article_dict[article_key]["journal"]
        text = article_dict[article_key]['text']
        if 'PLOS' in journal:
            article_type_regex = r'id="artType">(.+?)<.*'
            article_type = re.search(article_type_regex, text, re.DOTALL).group(1)
            types_to_exclude = ['Editorial', 'Correction', 'Perspective', 'Retraction']
            types_to_exclude += [type.lower() for type in types_to_exclude]
            types_to_exclude += [type.upper() for type in types_to_exclude]
        else:
            article_type = 'Research Article'
        if (verbose == True) or  ((article_key +1) - (article_key +1) //1 == 0): # if integer
            print(f'Journal: {journal} {article_key}')
            # print(f'\tArticle type: {article_type}')
        if (article_type == 'Research Article') or (article_type not in types_to_exclude):
            trimmed_text, display = trim_text(text, article_regex, abs_regex)
            text_dict[article_key] = {
                'title': article_dict[article_key]['title'],
                'body': f"{article_dict[article_key]['title']}\n\n{trimmed_text['body']}",
                'abstract': trimmed_text['abstract'],
            }
            if (to_display == 'all') or (to_display == None) or (article_key in to_display):
                display_dict[article_key] = {
                    'abstract': display['abs_display'],
                    'body': display['article_display']
                }
        else:
            print(f'\tArticle type "{article_type}" excluded')
    print(f'text_dict keys: {[key for key in text_dict.keys()]}')
    return text_dict, display_dict

def display_html(display_dict, type='abstract'):
    """
    Display the HTML from the dictionary of HTML displays.
    """
    print()
    for text in display_dict:
        print('************************************* Start *************************************')
        display.display(display_dict[text][type])


root_display_dict = dict()
text_dict = dict()
iteration = 2.2
text_id = 2

# regex = r'(.*)'
# text_dict[iteration], display_dict = text_dict_from_web(
#     partial_article_dict[text_id], to_display='all', header=(2,4), article_regex_str=regex, abs_regex_str=regex)

text_dict[iteration], display_dict = text_dict_from_web(
    partial_article_dict[text_id], to_display='all', header=(2,4), verbose=True)

root_display_dict[iteration] = display_dict
display_html(display_dict, type='abstract')



header: [234]
Regex patterns: 
	.*<h[234].*?>Abstract</h[234]>.*(?:(?:Introduction|Background).*)?(<h[234].*?>1?.?\s?(?:Introduction|Background)</h[234]>.*References)<.*
	.*(<h[234].*?>Abstract</h[234]>.*(?:(?:Introduction|Background).*)?)<h[234].*?>1?.?\s?(?:Introduction|Background)</h[234]>.*References<.*
Journal: BMJ Open 1.0
Journal: BMJ Open 1.01
Journal: BMJ Open 1.02
Journal: BMJ Open 1.03
Journal: BMJ Open 1.04
Journal: BMJ Open 1.05
Journal: BMJ Open 1.06
Journal: BMJ Open 1.07
Journal: BMJ Open 1.08
Journal: BMJ Open 1.09
Journal: BMJ Open 1.1
Journal: BMJ Open 1.11
Journal: BMJ Open 1.12
Journal: BMJ Open 1.13
Journal: BMJ Open 1.14
Journal: BMJ Open 1.15
Journal: BMJ Open 1.16
Journal: BMJ Open 1.17
Journal: BMJ Open 1.18
Journal: BMJ Open 1.19
Journal: BMJ Open 1.2
Journal: BMJ Open 1.21
Journal: BMJ Open 1.22
Journal: BMJ Open 1.23
Journal: BMJ Open 1.24
Journal: BMJ Open 1.25
Journal: BMJ Open 1.26
Journal: BMJ Open 1.27
Journal: BMJ Open 1.28
Journal: BMJ Open 1.29
Jour

************************************* Start *************************************


************************************* Start *************************************


************************************* Start *************************************


************************************* Start *************************************


...

************************************* Start *************************************


************************************* Start *************************************


************************************* Start *************************************


************************************* Start *************************************


************************************* Start *************************************


# text_id = 4 Journal of Medical Internet Research

In [8]:
text_id = 4
partial_article_dict[text_id] = create_partial_article_dict(article_dict[0], n_articles=30, journals=2)

Keys for article_dict: [2.0, 2.01, 2.02, 2.03, 2.04, 2.05, 2.06, 2.07, 2.08, 2.09]
Journals:
	Journal of Medical Internet Research


## Exclude Viewpoints and Perspectives articles from Journal of Medical Internet Research
<a href="/themes/22" data-test="article-collection" aria-label="403 articles belongs to Viewpoints and Perspectives e-collection/theme issue" class="collection__link">
                        Viewpoints and Perspectives (403)
                    </a>

In [23]:
article_type_regex = r'class="collection__link">.+<.*'
article_type_regex = r'.*(class=.collection..?link.).*'
article_type_regex = r'(.*)'

text = partial_article_dict[text_id][2.08]['text']
re.search(article_type_regex, text, re.DOTALL).group(1)
# Unable to find Viewpoints and Perspectives because it is in an `<a>` tag

'\n<h2 tabindex="0" data-test="article-collection" class="h4 green-heading-underline width-fit-content">\n                    This paper is in the following\n                    <span class="collection__span">e-collection/theme issue:</span></h2>\n<p id="main-content" tabindex="0">\n                            Published on\n                            <time datetime="19.2023">19.6.2023\n                            </time>\n                            in\n                            <span data-test="issue-info"><a href="/2023/1" class="nuxt-link-active">\n                                    Vol 25\n                                    <!----> (2023)\n                                    <!----></a></span></p>\n<h2 class="info__hidden-title">\n                            Providing Self-Led Mental Health Support Through an Artificial Intelligence–Powered Chat Bot (Leora) to Meet the Demand of Mental Health Care\n                        </h2>\n<p tabindex="0" class="authors-for-screen-reader

# 3 time out if no match in a certain time

In [24]:
from IPython import display

def trim_text(text, article_regex=None, abs_regex=None):
    if article_regex==None:
        article_regex = '.*<h2>Abstract</h2>.*(?:(?:Introduction|Background).*)?(<h2.*?>\d?.?\s?(?:Introduction|Background)</h2>.*References)<.*' 
        abs_regex = '.*(<h2>Abstract</h2>.*(?:(?:Introduction|Background).*)?)<h2.*?>\d?.?\s?(?:Introduction|Background)</h2>.*References<.*' 
    try:
        body = re.search(article_regex, text, re.DOTALL).group(1)
        abstract = re.search(abs_regex, text, re.DOTALL).group(1)
    except Exception as error: 
        exc_type, exc_obj, tb = sys.exc_info()
        file = tb.tb_frame
        lineno = tb.tb_lineno
        filename = file.f_code.co_filename
        print(f'\tAn error occurred on line {lineno} in {filename}: {error}')    
        print('\t\tUnable to parse article text')
        body = text 
        abstract = text 
    try:
        article_display = display.HTML(body)
        abs_display = display.HTML(abstract)
    except Exception as error: 
        exc_type, exc_obj, tb = sys.exc_info()
        file = tb.tb_frame
        lineno = tb.tb_lineno
        filename = file.f_code.co_filename
        print(f'\tAn error occurred on line {lineno} in {filename}: {error}')    
        print('\t\tUnable to create HTML display')
        article_display = f'<p>{body}</p>'
        abs_display = f'<p>{abstract}</p>'
    processed_article = {
        'abstract': abstract,
        'body': body,
    }
    display_dict = {
        'article_display': article_display,
        'abs_display': abs_display
    }
    return processed_article, display_dict

def text_dict_from_web(article_dict, header=(2,4), to_display=0.01, verbose=False,
        article_regex_str=r'.*<h\d.*?>Abstract</h\d>.*(?:(?:Introduction|Background).*)?(<h\d.*?>1?.?\s?(?:Introduction|Background)</h\d>.*References)<.*',
        abs_regex_str=r'.*(<h\d.*?>Abstract</h\d>.*(?:(?:Introduction|Background).*)?)<h\d.*?>1?.?\s?(?:Introduction|Background)</h\d>.*References<.*'
        ):
    """
    Create a text dictionary from a dictionary containing web-scraped articles.

    Parameters:
        article_dict (dict): Values of each dictionary item are a dictionary representing the data from a 
            single article: 'url', 'text', and 'title'.

    Returns:
        text_dict: Dictionary where each item is a string of the text of an article, starting with the title.
    """
    if type(header) == int:
        header = str(header) 
    else :
        header = rf"[{''.join([str(h) for h in range(header[0], header[-1]+1)])}]"
    print(rf'header: {header}')
    article_regex_str = article_regex_str.replace('\d', header)
    abs_regex_str = abs_regex_str.replace('\d', header)
    article_regex = rf'{article_regex_str}'
    abs_regex = rf'{abs_regex_str}'
    print(f'Regex patterns: \n\t{article_regex}\n\t{abs_regex}')
    text_dict = dict()
    display_dict = dict()
    if (type(to_display) == int) or (type(to_display) == float):
        to_display = [to_display] 
    for article_key in article_dict:
        journal = article_dict[article_key]["journal"]
        text = article_dict[article_key]['text']
        if 'PLOS' in journal:
            article_type_regex = r'id="artType">(.+?)<.*'
            article_type = re.search(article_type_regex, text, re.DOTALL).group(1)
            types_to_exclude = ['Editorial', 'Correction', 'Perspective', 'Retraction']
            types_to_exclude += [type.lower() for type in types_to_exclude]
            types_to_exclude += [type.upper() for type in types_to_exclude]
        elif journal == 'Journal of Medical Internet Research':
            article_type_regex = r'<meta name="DC.Type" content="(.+?)" />'
            article_type = re.search(article_type_regex, text, re.DOTALL).group(1)
            types_to_exclude = ['Editorial', 'Correction', 'Perspective', 'Retraction']
            types_to_exclude += [type.lower() for type in types_to_exclude]
            types_to_exclude += [type.upper() for type in types_to_exclude]
        else:
            article_type = 'Research Article'
        if (verbose == True) or  ((article_key +1) - (article_key +1) //1 == 0): # if integer
            print(f'Journal: {journal} {article_key}')
            # print(f'\tArticle type: {article_type}')
        if (article_type == 'Research Article') or (article_type not in types_to_exclude):
            try:
                trimmed_text, display = trim_text(text, article_regex, abs_regex)
                text_dict[article_key] = {
                    'title': article_dict[article_key]['title'],
                    'body': f"{article_dict[article_key]['title']}\n\n{trimmed_text['body']}",
                    'abstract': trimmed_text['abstract'],
                }
                if (to_display == 'all') or (to_display == None) or (article_key in to_display):
                    display_dict[article_key] = {
                        'abstract': display['abs_display'],
                        'body': display['article_display']
                    }
            except TimeoutError as error:
                print('\t\t**Timeout error during parsing attempt**')
        else:
            print(f'\tArticle type "{article_type}" excluded')
    print(f'text_dict keys: {[key for key in text_dict.keys()]}')
    return text_dict, display_dict

def display_html(display_dict, type='abstract'):
    """
    Display the HTML from the dictionary of HTML displays.
    """
    print()
    for text in display_dict:
        print('************************************* Start *************************************')
        display.display(display_dict[text][type])

def timeout_handler(signum, frame):
    raise TimeoutError('Search timed out')

signal.signal(signal.SIGALRM, timeout_handler)
signal.alarm(5)  # Timeout duration in seconds

# root_display_dict = dict()
# text_dict = dict()
iteration = 3
text_id = 4

# regex = r'(.*)'
# text_dict[iteration], display_dict = text_dict_from_web(
#     partial_article_dict[text_id], to_display='all', header=(2,4), article_regex_str=regex, abs_regex_str=regex)

text_dict[iteration], display_dict = text_dict_from_web(
    partial_article_dict[text_id], to_display='all', header=(2,4), verbose=True)

root_display_dict[iteration] = display_dict
display_html(display_dict, type='abstract')



AttributeError: module 'signal' has no attribute 'SIGALRM'

In [ ]:
9

# *End of Page*